In [26]:
import pandas as pd

df = pd.read_csv('csv_outputs/latest_tvl_app_trends_by_token.csv')



In [30]:
name = 'SushiSwap'
chain = 'Arbitrum'
trailing_num_days = 30

In [31]:
ndf = df[
        (df['name'].str.lower() == name.lower())
        & (df['chain'].str.lower() == chain.lower())
        # & (df['chain'].str.lower() == chain.lower())
]
# create a new column with ranks of "dt"
ndf['date_rank'] = ndf['date'].rank(method='min', ascending=False)

ndf = ndf[ndf['date_rank'] <= trailing_num_days]
# display(ndf)
# print(ndf.columns)

Index(['Unnamed: 0', 'date', 'token', 'chain', 'protocol', 'name', 'category',
       'parent_protocol', 'token_value', 'usd_value', 'total_app_tvl',
       'last_token_value', 'price_usd', 'last_price_usd', 'token_rank_desc',
       'token_rank_desc_prot', 'token_rank_desc_prot_gt0', 'latest_price_usd',
       'net_token_flow', 'net_dollar_flow', 'net_dollar_flow_latest_price',
       'date_rank'],
      dtype='object')


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_81505/166213775.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['date_rank'] = ndf['date'].rank(method='min', ascending=False)


In [33]:
piv_df = ndf.groupby(['chain','protocol','token']).agg({'net_dollar_flow_latest_price':'sum'})

display(piv_df)

,,,net_dollar_flow_latest_price
chain,protocol,token,
Arbitrum,sushiswap*,totalLiquidityUSD,4.511007e+07
